# Multiword expressions identification and extraction
The task shows two simple methods useful for identifying multiword expressions (MWE) in corpora.

## Use SpaCy tokenizer API to tokenize the text from the law corpus.

In [233]:
!pip install spacy
!pip install tabulate

In [234]:
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
from tqdm import tqdm
import sys
import os
from collections import Counter
import re
from numpy import log
from tabulate import tabulate

In [132]:
ustawy_path = '../ustawy'

In [133]:
nlp = Polish()
tokenizer = nlp.tokenizer

In [180]:
tokens_map = {}

with tqdm(total=1179, file=sys.stdout) as pbar: 
    for filename in os.listdir(ustawy_path):
            with open(ustawy_path + '/' + filename, 'r', encoding='utf8') as f:
                content = f.read()
                tokens = [token.text.strip().lower() for token in tokenizer(content)]
                tokens = list(filter(lambda x: x != '', tokens))
                tokens_map[filename] = Counter(tokens)
                pbar.update(1)

100%|██████████████████████████████████████████████████████████████████████████████| 1179/1179 [00:24<00:00, 48.06it/s]


In [181]:
list(tokens_map['1993_599.txt'].items())[:10]

[('dz', 4),
 ('.', 497),
 ('u', 9),
 ('z', 108),
 ('1993', 9),
 ('r', 20),
 ('nr', 22),
 ('129', 2),
 (',', 306),
 ('poz', 17)]

In [182]:
def isalpha(s):
    for letter in s:
        if not letter.isalpha():
            return False
    return True 

In [183]:
counter_total = Counter()

for _, counter in tokens_map.items():
    counter_total += counter

In [184]:
unigrams_total = Counter({x: count for x, count in counter_total.items() if isalpha(x)})

In [185]:
list(unigrams_total.items())[:10]

[('dz', 8885),
 ('u', 9153),
 ('z', 82443),
 ('r', 33177),
 ('nr', 44950),
 ('poz', 45224),
 ('ustawa', 3235),
 ('dnia', 17954),
 ('grudnia', 2117),
 ('o', 64776)]

In [186]:
bigrams_map = {}

for filename, tokens in tokens_map.items():
    bigrams_map[filename] = []
    previous_token = None
    for token, _ in tokens.items():
        if previous_token is not None:
            bigrams_map[filename].append((previous_token, token))
        previous_token = token
    bigrams_map[filename] = Counter(bigrams_map[filename])

## Compute bigram counts of downcased tokens.

In [187]:
bigrams_total = Counter()

with tqdm(total=1179, file=sys.stdout) as pbar: 
    for _, counter in bigrams_map.items():
        bigrams_total += counter
        pbar.update(1)

100%|██████████████████████████████████████████████████████████████████████████████| 1179/1179 [00:17<00:00, 67.11it/s]


In [188]:
list(bigrams_total.items())[:10]

[(('dz', '.'), 946),
 (('.', 'u'), 946),
 (('u', 'z'), 946),
 (('z', '1993'), 8),
 (('1993', 'r'), 8),
 (('r', 'nr'), 946),
 (('nr', '129'), 11),
 (('129', ','), 11),
 ((',', 'poz'), 1032),
 (('poz', '599'), 1)]

## Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries after computing the bigram counts.

In [189]:
bigrams_total = Counter({x: count for x, count in bigrams_total.items() if isalpha(x[0]) and isalpha(x[1])})

In [190]:
list(bigrams_total.items())[:10]

[(('u', 'z'), 946),
 (('r', 'nr'), 946),
 (('ustawa', 'dnia'), 820),
 (('grudnia', 'o'), 78),
 (('o', 'zmianie'), 695),
 (('zmianie', 'ustawy'), 505),
 (('ustawy', 'podatku'), 31),
 (('podatku', 'od'), 25),
 (('od', 'towarów'), 25),
 (('towarów', 'i'), 17)]

In [191]:
total = sum(unigrams_total.values())
total

3579380

## Use pointwise mutual information to compute the measure for all pairs of words.

In [210]:
def p(word_occ):
    return word_occ / total

def pmi2(word1, word2):
    word1_occ = unigrams_total[word1]
    word2_occ = unigrams_total[word2]
    together_occ = bigrams_total[(word1, word2)]
    return log(p(together_occ)) - log(p(word1_occ)) - log(p(word2_occ))

In [215]:
bigrams_total_pmi = {bigram: pmi2(bigram[0], bigram[1]) for bigram, _  in bigrams_total.items()}

## Sort the word pairs according to that measure in the descending order and determine top 10 entries.

In [216]:
Counter(bigrams_total_pmi).most_common()[:10]

[(('kołowe', 'jednoosiowe'), 15.090700159021198),
 (('jednoosiowe', 'dwuosiowe'), 15.090700159021198),
 (('pieluchy', 'pielucho'), 15.090700159021198),
 (('witaminizowane', 'talki'), 15.090700159021198),
 (('kijki', 'nart'), 15.090700159021198),
 (('ceratki', 'gryzaczki'), 15.090700159021198),
 (('mundurowe', 'zuchów'), 15.090700159021198),
 (('zuchów', 'harcerzy'), 15.090700159021198),
 (('zbiornikowe', 'wyłą'), 15.090700159021198),
 (('zbrojeń', 'żelbeto'), 15.090700159021198)]

## Filter bigrams with number of occurrences lower than 5. Determine top 10 entries for the remaining dataset (>=5 occurrences).

In [217]:
bigrams_filtered_pmi = {bigram: pmi2(bigram[0], bigram[1]) for bigram, count in bigrams_total.items() if count >= 5}

In [228]:
bigrams_pmi_top10 = Counter(bigrams_filtered_pmi).most_common()[:10]
bigrams_pmi_top10

[(('świeckie', 'przygotowujące'), 13.481262246587098),
 (('brat', 'siostra'), 13.481262246587098),
 (('grzegorz', 'schetyna'), 13.481262246587098),
 (('krewnym', 'powinowatym'), 13.144790009965885),
 (('klęską', 'żywiołową'), 13.144790009965885),
 (('chwytów', 'obezwładniających'), 13.144790009965885),
 (('podrobiony', 'przerobiony'), 12.96246845317193),
 (('chrześcijan', 'baptystów'), 12.80831777334467),
 (('zapieczętowanej', 'kopercie'), 12.80831777334467),
 (('nadmiernymi', 'trudnościami'), 12.788115066027153)]

## Use log likelihood ratio (LLR) to compute the measure for all pairs of words.

In [197]:
total_bigrams = sum(bigrams_total.values())

In [198]:
def H(counts):
    total = float(sum(counts))
    return sum([k * log(k / total + (k==0)) for k in counts])


def llr2(word1, word2):
    together_occ = bigrams_total[(word1, word2)]
    k11 = together_occ
    k12 = unigrams_total[word2] - together_occ
    k21 = unigrams_total[word1] - together_occ
    k22 = total_bigrams - k12 - k21 - k11
    
    return 2 * (H([k11, k12, k21, k22]) - H([k11 + k12, k21 + k22]) - H([k11 + k21, k12 + k22]))

In [199]:
bigrams_total_llr = {bigram: llr2(bigram[0], bigram[1]) for bigram, _  in bigrams_total.items()}

## Sort the word pairs according to that measure in the descending order and display top 10 entries.

In [227]:
bigrams_llr_top10 = Counter(bigrams_total_llr).most_common()[:10]
bigrams_llr_top10

[(('w', 'art'), 60644.41671135172),
 (('art', 'w'), 60453.14603600174),
 (('do', 'w'), 43015.97084866068),
 (('w', 'ust'), 37649.54294047924),
 (('lub', 'w'), 31994.111593838432),
 (('się', 'w'), 31953.025928710296),
 (('oraz', 'w'), 23182.058776851685),
 (('z', 'art'), 21935.846033818438),
 (('mowa', 'w'), 19655.880910842447),
 (('art', 'do'), 16076.934222189826)]

## Compute trigram counts for the whole corpus and perform the same filtering.

In [201]:
trigrams_map = {}

for filename, tokens in tokens_map.items():
    trigrams_map[filename] = []
    prev_token = None
    prev_prev_token = None
    for token, _ in tokens.items():
        if prev_token is not None and prev_prev_token is not None:
            trigrams_map[filename].append((prev_prev_token, prev_token, token))
        prev_prev_token = prev_token    
        prev_token = token
    trigrams_map[filename] = Counter(trigrams_map[filename])

In [202]:
trigrams_total = Counter()

with tqdm(total=1179, file=sys.stdout) as pbar: 
    for _, counter in trigrams_map.items():
        trigrams_total += counter
        pbar.update(1)

100%|██████████████████████████████████████████████████████████████████████████████| 1179/1179 [00:29<00:00, 39.52it/s]


In [203]:
list(trigrams_total.items())[:10]

[(('dz', '.', 'u'), 946),
 (('.', 'u', 'z'), 946),
 (('u', 'z', '1993'), 8),
 (('z', '1993', 'r'), 8),
 (('1993', 'r', 'nr'), 8),
 (('r', 'nr', '129'), 11),
 (('nr', '129', ','), 11),
 (('129', ',', 'poz'), 11),
 ((',', 'poz', '599'), 1),
 (('poz', '599', 'ustawa'), 1)]

In [204]:
trigrams_total = Counter({trigram: count for trigram, count in trigrams_total.items() if isalpha(trigram[0]) and isalpha(trigram[1]) and isalpha(trigram[2])})

In [205]:
list(trigrams_total.items())[:10]

[(('grudnia', 'o', 'zmianie'), 64),
 (('o', 'zmianie', 'ustawy'), 502),
 (('zmianie', 'ustawy', 'podatku'), 31),
 (('ustawy', 'podatku', 'od'), 16),
 (('podatku', 'od', 'towarów'), 18),
 (('od', 'towarów', 'i'), 15),
 (('towarów', 'i', 'usług'), 17),
 (('i', 'usług', 'oraz'), 14),
 (('usług', 'oraz', 'akcyzowym'), 15),
 (('oraz', 'akcyzowym', 'art'), 13)]

## Use PMI (with 5 occurrence threshold) and LLR to compute top 10 results for the trigrams. Devise a method for computing the values, based on the results for bigrams.

In [206]:
total_trigrams = sum(trigrams_total.values())

In [207]:
def llr3(word1, word2, word3):
    together_occ = trigrams_total[(word1, word2, word3)]
    k11 = together_occ
    k12 = bigrams_total[(word2, word3)] - together_occ
    k21 = bigrams_total[(word1, word2)] - together_occ
    k22 = total_trigrams - (k11 + k12 + k21)
    return 2 * (H([k11, k12, k21, k22]) - H([k11 + k12, k21 + k22]) - H([k11 + k21, k12 + k22]))

In [208]:
trigrams_total_llr = {trigram: llr3(trigram[0], trigram[1], trigram[2]) for trigram, _  in trigrams_total.items()}

In [225]:
trigrams_llr_top10 = Counter(trigrams_total_llr).most_common()[:10]
trigrams_llr_top10

[(('wprowadza', 'się', 'następujące'), 8915.192356069196),
 (('się', 'następujące', 'zmiany'), 8608.4501027276),
 (('o', 'zmianie', 'ustawy'), 7348.741284552849),
 (('dni', 'od', 'ogłoszenia'), 3787.3696206025415),
 (('ustawa', 'z', 'dnia'), 3783.0820706745662),
 (('określi', 'drodze', 'rozporządzenia'), 3518.7683073980124),
 (('życie', 'po', 'upływie'), 3395.511101729391),
 (('przepisy', 'ogólne', 'art'), 3238.6399183346157),
 (('minister', 'właściwy', 'spraw'), 2878.3895869530506),
 (('wchodzi', 'życie', 'po'), 2770.397145362789)]

In [219]:
def pmi3(word1, word2, word3):
    word1_occ = unigrams_total[word1]
    word2_occ = unigrams_total[word2]
    word3_occ = unigrams_total[word3]
    together_occ = trigrams_total[(word1, word2, word3)]
    return log(p(together_occ)) - log(p(word1_occ)) - log(p(word2_occ)) - log(p(word3_occ))

In [220]:
trigrams_filtered_pmi = {trigram: pmi3(trigram[0], trigram[1], trigram[2]) for trigram, count in trigrams_total.items() if count >= 5}

In [224]:
trigrams_pmi_top10 = Counter(trigrams_filtered_pmi).most_common()[:10]
trigrams_pmi_top10

[(('estońskiej', 'cypryjskiej', 'łotewskiej'), 25.430276219431228),
 (('cypryjskiej', 'łotewskiej', 'litewskiej'), 25.3249157037734),
 (('brytanii', 'irlandii', 'północnej'), 25.316947534124225),
 (('ziemiach', 'zachodnich', 'północnych'), 25.180815359799645),
 (('łotewskiej', 'litewskiej', 'węgierskiej'), 25.06255143930591),
 (('przedwczesnego', 'wyrębu', 'drzewostanu'), 24.923904946014616),
 (('prostej', 'brat', 'siostra'), 24.908400759478653),
 (('czeskiej', 'estońskiej', 'cypryjskiej'), 24.76529991583798),
 (('litewskiej', 'węgierskiej', 'malty'), 24.35835442255938),
 (('wielkiej', 'brytanii', 'irlandii'), 24.218335245456114)]


## Create a table comparing the methods (separate table for bigrams and trigrams).

In [230]:
def extract_ngram(result):
    return list(map(lambda x: x[0], result))

In [231]:
extract_ngram(bigrams_pmi_top10)

[('świeckie', 'przygotowujące'),
 ('brat', 'siostra'),
 ('grzegorz', 'schetyna'),
 ('krewnym', 'powinowatym'),
 ('klęską', 'żywiołową'),
 ('chwytów', 'obezwładniających'),
 ('podrobiony', 'przerobiony'),
 ('chrześcijan', 'baptystów'),
 ('zapieczętowanej', 'kopercie'),
 ('nadmiernymi', 'trudnościami')]

### Table for bigrams

In [236]:
bigram_table = {'PMI': extract_ngram(bigrams_pmi_top10), 'LLR': extract_ngram(bigrams_llr_top10)}
print(tabulate(bigram_table, headers='keys', tablefmt='fancy_grid'))

╒══════════════════════════════════╤═══════════════╕
│ PMI                              │ LLR           │
╞══════════════════════════════════╪═══════════════╡
│ ('świeckie', 'przygotowujące')   │ ('w', 'art')  │
├──────────────────────────────────┼───────────────┤
│ ('brat', 'siostra')              │ ('art', 'w')  │
├──────────────────────────────────┼───────────────┤
│ ('grzegorz', 'schetyna')         │ ('do', 'w')   │
├──────────────────────────────────┼───────────────┤
│ ('krewnym', 'powinowatym')       │ ('w', 'ust')  │
├──────────────────────────────────┼───────────────┤
│ ('klęską', 'żywiołową')          │ ('lub', 'w')  │
├──────────────────────────────────┼───────────────┤
│ ('chwytów', 'obezwładniających') │ ('się', 'w')  │
├──────────────────────────────────┼───────────────┤
│ ('podrobiony', 'przerobiony')    │ ('oraz', 'w') │
├──────────────────────────────────┼───────────────┤
│ ('chrześcijan', 'baptystów')     │ ('z', 'art')  │
├──────────────────────────────────┼──────────

### Table for trigrams

In [237]:
trigram_table = {'PMI': extract_ngram(trigrams_pmi_top10), 'LLR': extract_ngram(trigrams_llr_top10)}
print(tabulate(trigram_table, headers='keys', tablefmt='fancy_grid'))

╒═════════════════════════════════════════════╤═════════════════════════════════════════╕
│ PMI                                         │ LLR                                     │
╞═════════════════════════════════════════════╪═════════════════════════════════════════╡
│ ('estońskiej', 'cypryjskiej', 'łotewskiej') │ ('wprowadza', 'się', 'następujące')     │
├─────────────────────────────────────────────┼─────────────────────────────────────────┤
│ ('cypryjskiej', 'łotewskiej', 'litewskiej') │ ('się', 'następujące', 'zmiany')        │
├─────────────────────────────────────────────┼─────────────────────────────────────────┤
│ ('brytanii', 'irlandii', 'północnej')       │ ('o', 'zmianie', 'ustawy')              │
├─────────────────────────────────────────────┼─────────────────────────────────────────┤
│ ('ziemiach', 'zachodnich', 'północnych')    │ ('dni', 'od', 'ogłoszenia')             │
├─────────────────────────────────────────────┼─────────────────────────────────────────┤
│ ('łotews

### Why do we have to filter the bigrams, rather than the token sequence?

Bigrams were created before cleaning tokens from not containing letters, so we received more meaningful collocations.

### Which measure (PMI, PMI with filtering, LLR) works better for the bigrams and which for the trigrams?
In my opinion LLR works better for both the bigrams and the trigrams (more meaningful collocations found).

PMI with filtering gave better results than the one without.

### What types of expressions are discovered by the methods.
- prepositional phrase 
- epithet
- etc.

### Can you devise a different type of filtering that would yield better results?
Filter with parts of speech (for example get rid of 'czeskiej estońskiej cypryjskiej', which is not interesting).
